In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.imagenet_utils import preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexEntropy"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = VGG16(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 512)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 512)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.3min


[CV] ........................... C=0.01, score=0.930528, total= 1.3min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.922244, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.920319, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.914062, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.919483, total= 1.4min
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.896654, total= 1.4min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.925595, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.6min


[CV] ............................ C=0.1, score=0.920714, total= 1.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.909180, total= 1.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.924213, total= 1.6min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.916501, total= 1.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.906034, total= 1.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.901961, total= 1.7min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.920319, total= 1.7min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.915339, total= 2.0min
[CV] ................... C=1000000000.0, score=0.923679, total= 2.0min
[CV] ................... C=1000000000.0, score=0.900591, total= 1.8min
[CV] ................... C=1000000000.0, score=0.879327, total= 1.8min
[CV] ................... C=1000000000.0, score=0.904762, total= 1.8min
[CV] ................... C=1000000000.0, score=0.911794, total= 1.8min
[CV] ................... C=1000000000.0, score=0.914370, total= 1.9min
[CV] ................... C=1000000000.0, score=0.907555, total= 1.8min
[CV] ................... C=1000000000.0, score=0.915339, total= 1.8min
[CV] .................. C=10000000000.0, score=0.912109, total= 1.6min
[CV] .................. C=10000000000.0, score=0.892368, total= 1.5min
[CV] .................. C=10000000000.0, score=0.914706, total= 1.5min
[CV] .................. C=10000000000.0, score=0.899606, total= 1.5min
[CV] .................. C=10000000000.0, score=0.902077, total= 1.5min
[CV] .

[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 10.0min remaining:   23.9s


[CV] .................. C=10000000000.0, score=0.918307, total= 1.5min
[CV] .................. C=10000000000.0, score=0.912525, total= 1.4min
[CV] .................. C=10000000000.0, score=0.895937, total= 1.5min
[CV] .................. C=10000000000.0, score=0.912351, total= 1.4min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 10.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 84.00454004,  99.28956895, 104.57067804, 106.3615958 ,
        110.72510722, 105.14131927, 111.89341509, 107.19652762,
        110.7806293 , 110.65800371, 106.16843193, 109.96862559,
         90.11670098]),
 'mean_score_time': array([0.0297292 , 0.03651829, 0.02908871, 0.01784997, 0.01988163,
        0.01765482, 0.02339444, 0.01903465, 0.01497295, 0.01924679,
        0.0180192 , 0.01408405, 0.01308215]),
 'mean_test_score': array([0.9210734 , 0.91831097, 0.91051697, 0.91110892, 0.90568272,
        0.90499211, 0.91416732, 0.90745856, 0.90420284, 0.898382  ,
        0.90893844, 0.90686661, 0.90883978]),
 'mean_train_score': array([0.961019  , 0.97653013, 0.97340706, 0.97476684, 0.96697095,
        0.96458781, 0.97537813, 0.97114962, 0.96540187, 0.96046127,
        0.96844249, 0.96882557, 0.97013604]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92107, std: 0.00547, params: {'C': 0.01},
 mean: 0.91831, std: 0.00813, params: {'C': 0.1},
 mean: 0.91052, std: 0.00886, params: {'C': 1.0},
 mean: 0.91111, std: 0.00861, params: {'C': 10.0},
 mean: 0.90568, std: 0.00832, params: {'C': 100.0},
 mean: 0.90499, std: 0.01766, params: {'C': 1000.0},
 mean: 0.91417, std: 0.00522, params: {'C': 10000.0},
 mean: 0.90746, std: 0.00655, params: {'C': 100000.0},
 mean: 0.90420, std: 0.01099, params: {'C': 1000000.0},
 mean: 0.89838, std: 0.02029, params: {'C': 10000000.0},
 mean: 0.90894, std: 0.00828, params: {'C': 100000000.0},
 mean: 0.90687, std: 0.01119, params: {'C': 1000000000.0},
 mean: 0.90884, std: 0.01058, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9211
